## Entendendo como extrair informações sobre repositórios através da api do GitHub

Links: 
- [Search](https://developer.github.com/v3/search/)
- [Searching for repositories](https://help.github.com/en/articles/searching-for-repositories#search-by-repository-name-description-or-contents-of-the-readme-file)
- Lista de qualifiers:  https://help.github.com/en/articles/searching-code
- Documentação para search: https://developer.github.com/v3/search/

In [72]:
import requests
import pandas as pd
import time
from datetime import datetime

Montando url de busca para os repositórios com maior quantidade de estrelas.
Usando como consulta stars:>1 para indicar que são todos os repositorios acima de 1 estrela, o que não tem impacto negativo para a busca pois existem certa de 3.305.358 repositórios e só vamos analisar 2.500.

In [138]:
q = 'q=stars:>1'
sort = '&sort=stars&order=desc'
url_base = 'https://api.github.com/search/repositories?'
url_final = url_base+q+sort
credentials=('lorenaps','token')
url_final

'https://api.github.com/search/repositories?q=stars:>1&sort=stars&order=desc'

Verificando o limite de requisições

In [94]:
t = requests.get('https://api.github.com/rate_limit', auth=credentials)
t.json()

{'rate': {'limit': 5000, 'remaining': 5000, 'reset': 1562436829},
 'resources': {'core': {'limit': 5000, 'remaining': 5000, 'reset': 1562436829},
  'graphql': {'limit': 5000, 'remaining': 5000, 'reset': 1562436829},
  'integration_manifest': {'limit': 5000,
   'remaining': 5000,
   'reset': 1562436829},
  'search': {'limit': 30, 'remaining': 30, 'reset': 1562433289}}}

In [87]:
colunas=['id',
         'owner_type', 
           'owner_url',
           'owner_html_url',
           'html_url',
           'url',
           'fork',
           'created_at',
           'updated_at',
           'size',
           'stargazers_count',
           'language',
           'has_issues',
           'has_wiki',
           'forks_count',
           'forks',
           'open_issues',
           'watchers',
         'timestamp_extract']

In [127]:
def add_resultado(item):
    df = pd.DataFrame([[
                        item.get('id'),
                        item.get('owner').get('type', None),
                        item.get('owner').get('url', None),
                        item.get('owner').get('html_url', None),
                        item.get('html_url', None),
                        item.get('url', None),
                        item.get('fork', None),
                        item.get('created_at', None),
                        item.get('updated_at', None),
                        item.get('size', None),
                        item.get('stargazers_count', None),
                        item.get('language', None),
                        item.get('has_issues', None),
                        item.get('has_wiki', None),
                        item.get('forks_count', None),
                        item.get('forks', None),
                        item.get('open_issues', None),
                        item.get('watchers', None),
                        str(time.time()).split('.')[0]]], columns=colunas)

    return df    

In [128]:
def extrair_dados(data, resultados):
    for item in data.get('items', None):
        resultados = pd.concat([resultados, add_resultado(item)], ignore_index=True, sort=False)
        
    return resultados

In [129]:
def get_total_paginas(data):
    
    itens_por_pagina = len(data.get('items'))
    total_paginas = data.get('total_count') // itens_por_pagina # opreração de div em python
    total_paginas

    print('Total de registros:{0} , Registros por página:{1}, Total de Páginas:{2}'.format(
        data.get('total_count'), itens_por_pagina, total_paginas))
    
    return total_paginas        

Para pegar em torno de 2500 registros, semelhante ao artigo original, seria preciso 84 iterações.
Mas como a api limita o ferramenta de pesquisa aos 1000 primeiros resultados, usaremos 34 iterações.
É possíve fazer o teste requisitanto a página 34 dos resultados

In [130]:
mais_de_mil = 'https://api.github.com/search/repositories?q=stars%3A%3E1&sort=stars&order=desc&page=35'
t = requests.get(mais_de_mil, auth=credentials)
t.json()

{'documentation_url': 'https://developer.github.com/v3/search/',
 'message': 'Only the first 1000 search results are available'}

In [131]:
def percorrendo_paginas(url):
    
    resultados = pd.DataFrame(columns=['id', 
                                       'owner_type', 
                                   'owner_url',
                                   'owner_html_url',
                                   'html_url',
                                   'url',
                                   'fork',
                                   'created_at',
                                   'updated_at',
                                   'size',
                                   'stargazers_count',
                                   'language',
                                   'has_issues',
                                   'has_wiki',
                                   'forks_count',
                                   'forks',
                                   'open_issues',
                                   'watchers',
                                   'timestamp_extract',
                                   'commits',
                                   'contributors',])
    
    print('Extraindo página:1')
    results = requests.get(url, auth=credentials)    
    data = dict(results.json())
    resultados = extrair_dados(data, resultados)

    iteracoes = 34 
    
    for iteracao in range(1, iteracoes):
        print("\n>>>>>>>> Iteracao:{0}".format(iteracao+1))
        print("Tempo atual:{0}".format(datetime.now()))

        # Para requisições não autenticadas a api restringe para 10 requisições por minuto,
        # Para requisições autenticadas 30 por minuto.
        #if iteracao % 10 == 0:
        #    print("sleep 1 minuto")
        #    time.sleep(60)
        
        header = dict(results.links)
        next_url = header.get('next').get('url')
        print("Next url extraída: {0}".format(next_url))
        
        print('Extraindo página:{0}'.format(iteracao+1))
        results = requests.get(next_url, auth=credentials)
        print('Status:{0}'.format(results))
        
        data = dict(results.json())
            
        resultados = extrair_dados(data, resultados)
        
    return resultados

In [132]:
str(time.time()).split('.')[0]

'1562435050'

In [133]:
%%time
resultados = percorrendo_paginas(url_final)

Extraindo página:1

>>>>>>>> Iteracao:2
Tempo atual:2019-07-06 14:44:12.723108
Next url extraída: https://api.github.com/search/repositories?q=stars%3A%3E1&sort=stars&order=desc&page=2
Extraindo página:2
Status:<Response [200]>

>>>>>>>> Iteracao:3
Tempo atual:2019-07-06 14:44:14.959309
Next url extraída: https://api.github.com/search/repositories?q=stars%3A%3E1&sort=stars&order=desc&page=3
Extraindo página:3
Status:<Response [200]>

>>>>>>>> Iteracao:4
Tempo atual:2019-07-06 14:44:17.274794
Next url extraída: https://api.github.com/search/repositories?q=stars%3A%3E1&sort=stars&order=desc&page=4
Extraindo página:4
Status:<Response [200]>

>>>>>>>> Iteracao:5
Tempo atual:2019-07-06 14:44:19.055233
Next url extraída: https://api.github.com/search/repositories?q=stars%3A%3E1&sort=stars&order=desc&page=5
Extraindo página:5
Status:<Response [200]>

>>>>>>>> Iteracao:6
Tempo atual:2019-07-06 14:44:20.865714
Next url extraída: https://api.github.com/search/repositories?q=stars%3A%3E1&sort=sta

In [135]:
resultados.describe()

,id,owner_type,owner_url,owner_html_url,html_url,url,fork,created_at,updated_at,size,...,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,timestamp_extract,commits,contributors
count,1020,1020,1020,1020,1020,1020,1020,1020,1020,1020,...,902,1020,1020,1020,1020,1020,1020,1020,0.0,0.0
unique,1020,2,859,859,1020,1020,1,1020,1002,1003,...,40,2,2,949,949,479,989,40,0.0,0.0
top,3214406,Organization,https://api.github.com/users/facebook,https://github.com/facebook,https://github.com/gionkunz/chartist-js,https://api.github.com/repos/sindresorhus/awes...,False,2010-03-18T22:32:22Z,2019-07-06T15:32:45Z,284,...,JavaScript,True,True,1203,1203,2,12466,1562435071,NaN,NaN
freq,1,621,17,17,1,1,1020,1,2,2,...,327,974,685,3,3,18,2,30,NaN,NaN


In [136]:
resultados.to_csv('../dados/repositorios_06_07_2019.csv', index=False)

In [137]:
resultados.columns.values

array(['id', 'owner_type', 'owner_url', 'owner_html_url', 'html_url',
       'url', 'fork', 'created_at', 'updated_at', 'size',
       'stargazers_count', 'language', 'has_issues', 'has_wiki',
       'forks_count', 'forks', 'open_issues', 'watchers',
       'timestamp_extract', 'commits', 'contributors'], dtype=object)

## Extraindo commits e contributors através da biblioteca PyGithub

Links:
- https://github.com/PyGithub/PyGithub

In [139]:
from github import Github
from github import Repository
from github import ContentFile
from bs4 import BeautifulSoup as bs

In [140]:
token = '33449740798fb62fa5f9117251bc0a00385f34de'

In [141]:
g = Github(token)

In [143]:
resultados_csv = pd.read_csv('../dados/repositorios_06_07_2019.csv')
resultados_csv.head()

,id,owner_type,owner_url,owner_html_url,html_url,url,fork,created_at,updated_at,size,...,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,timestamp_extract,commits,contributors
0,28457823,Organization,https://api.github.com/users/freeCodeCamp,https://github.com/freeCodeCamp,https://github.com/freeCodeCamp/freeCodeCamp,https://api.github.com/repos/freeCodeCamp/free...,False,2014-12-24T17:49:19Z,2019-07-06T17:41:39Z,117553,...,JavaScript,True,False,22196,22196,1433,303703,1562435052,NaN,NaN
1,177736533,User,https://api.github.com/users/996icu,https://github.com/996icu,https://github.com/996icu/996.ICU,https://api.github.com/repos/996icu/996.ICU,False,2019-03-26T07:31:14Z,2019-07-06T15:23:35Z,59169,...,Rust,False,False,21325,21325,16696,246419,1562435052,NaN,NaN
2,11730342,Organization,https://api.github.com/users/vuejs,https://github.com/vuejs,https://github.com/vuejs/vue,https://api.github.com/repos/vuejs/vue,False,2013-07-29T03:24:51Z,2019-07-06T17:33:13Z,27671,...,JavaScript,True,True,20674,20674,312,143036,1562435052,NaN,NaN
3,2126244,Organization,https://api.github.com/users/twbs,https://github.com/twbs,https://github.com/twbs/bootstrap,https://api.github.com/repos/twbs/bootstrap,False,2011-07-29T21:19:00Z,2019-07-06T16:37:43Z,143925,...,JavaScript,True,False,65975,65975,366,134418,1562435052,NaN,NaN
4,10270250,Organization,https://api.github.com/users/facebook,https://github.com/facebook,https://github.com/facebook/react,https://api.github.com/repos/facebook/react,False,2013-05-24T16:15:54Z,2019-07-06T16:14:15Z,143121,...,JavaScript,True,True,24471,24471,751,132169,1562435052,NaN,NaN


In [26]:
resultados_csv[355:]

,id,owner_type,owner_url,owner_html_url,html_url,url,fork,created_at,updated_at,size,...,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,commits,contributors,readme
355,90528830,User,https://api.github.com/users/Solido,https://github.com/Solido,https://github.com/Solido/awesome-flutter,https://api.github.com/repos/Solido/awesome-fl...,False,2017-05-07T11:45:27Z,2019-07-01T19:40:07Z,1372,...,Dart,False,False,2428,2428,27,19302,1189.0,173,NaN
356,18840003,Organization,https://api.github.com/users/kriasoft,https://github.com/kriasoft,https://github.com/kriasoft/react-starter-kit,https://api.github.com/repos/kriasoft/react-st...,False,2014-04-16T13:08:18Z,2019-07-01T14:13:06Z,4622,...,JavaScript,True,True,3879,3879,486,19294,NaN,NaN,NaN
357,4037197,Organization,https://api.github.com/users/ycm-core,https://github.com/ycm-core,https://github.com/ycm-core/YouCompleteMe,https://api.github.com/repos/ycm-core/YouCompl...,False,2012-04-16T03:12:14Z,2019-07-01T18:53:40Z,34969,...,Python,True,True,2153,2153,52,19293,NaN,NaN,NaN
358,19141383,User,https://api.github.com/users/fouber,https://github.com/fouber,https://github.com/fouber/blog,https://api.github.com/repos/fouber/blog,False,2014-04-25T09:44:42Z,2019-07-01T10:27:13Z,19107,...,NaN,True,True,2186,2186,11,19263,NaN,NaN,NaN
359,60325062,User,https://api.github.com/users/terryum,https://github.com/terryum,https://github.com/terryum/awesome-deep-learni...,https://api.github.com/repos/terryum/awesome-d...,False,2016-06-03T06:48:30Z,2019-07-01T17:50:13Z,147,...,TeX,True,True,3703,3703,28,19196,NaN,NaN,NaN
360,8859474,User,https://api.github.com/users/skylot,https://github.com/skylot,https://github.com/skylot/jadx,https://api.github.com/repos/skylot/jadx,False,2013-03-18T17:08:21Z,2019-07-01T14:11:30Z,13473,...,Java,True,True,2108,2108,90,19137,NaN,NaN,NaN
361,6007295,User,https://api.github.com/users/fxsjy,https://github.com/fxsjy,https://github.com/fxsjy/jieba,https://api.github.com/repos/fxsjy/jieba,False,2012-09-29T07:52:01Z,2019-07-01T16:22:31Z,44251,...,Python,True,True,4969,4969,482,19127,NaN,NaN,NaN
362,12244426,User,https://api.github.com/users/rstacruz,https://github.com/rstacruz,https://github.com/rstacruz/nprogress,https://api.github.com/repos/rstacruz/nprogress,False,2013-08-20T13:58:02Z,2019-07-01T14:19:58Z,344,...,JavaScript,True,True,1588,1588,114,19122,NaN,NaN,NaN
363,790359,Organization,https://api.github.com/users/sequelize,https://github.com/sequelize,https://github.com/sequelize/sequelize,https://api.github.com/repos/sequelize/sequelize,False,2010-07-22T07:11:11Z,2019-07-01T17:54:03Z,25711,...,JavaScript,True,False,3005,3005,537,19120,NaN,NaN,NaN
364,33895378,User,https://api.github.com/users/bevacqua,https://github.com/bevacqua,https://github.com/bevacqua/dragula,https://api.github.com/repos/bevacqua/dragula,False,2015-04-13T21:35:38Z,2019-07-01T12:56:36Z,1832,...,JavaScript,True,False,1639,1639,212,19092,NaN,NaN,NaN


In [145]:
id_repos = resultados_csv.id
len(id_repos)

1020

In [148]:
id_repos

0        28457823
1       177736533
2        11730342
3         2126244
4        10270250
5        45717250
6        13491895
7        21737465
8        14440270
9          291137
10        6498492
11        1062897
12         943149
13       85077558
14       60493101
15       41881900
16       29028775
17        2325298
18        9384267
19      121395510
20       21289110
21       63537249
22       31792824
23       83222441
24       27193779
25       23088740
26        2561582
27       23096959
28        3470471
29       35955666
          ...    
990      26784827
991      73929422
992      32689863
993      28704549
994        908607
995      22553797
996     103749180
997      14376285
998      19463625
999     189890377
1000      2609642
1001       312262
1002     11393110
1003     53632140
1004       138312
1005      2826918
1006     25449064
1007       331603
1008     45936895
1009     47561138
1010     18405734
1011     27835638
1012    138331573
1013     41215439
1014    12

In [146]:
def get_contributors(url_repo):
    page = requests.get(url_repo)
    page_parse = bs(page.text, 'html.parser')
    lista = page_parse.findAll('span', attrs={'class': 'num text-emphasized'}) 
    
    text = lista[3].text
    contributors = text.replace(" ", "").replace("\n", "").replace(",", "")
    #contributors = int(contributors)
    
    return(contributors)

In [150]:
it = 0
for id_repo in id_repos:
    
    it = it + 1
    print("\nIteração: ", it)
    
    #if it % 10 == 0:
    #    print("sleep 1 minuto")
    #    time.sleep(60)

    repo = g.get_repo(id_repo)
    print("Nome do repositório:{0}".format(repo.name))

    commits = repo.get_commits().totalCount
    print("Commits: ", commits)

    url_repo = resultados_csv.loc[resultados_csv["id"] == id_repo]['html_url'].to_list()[0]
    print("Url: ", url_repo)

    contributors = get_contributors(url_repo)
    print("Contributors: ", contributors)

    print("Atualizando data frame :)")
    resultados_csv.loc[resultados_csv["id"] == id_repo, 'commits'] = commits 
    resultados_csv.loc[resultados_csv["id"] == id_repo, 'contributors'] = contributors

    resultados_csv.to_csv('../dados/repositorios_06_07_2019_update.csv', index=False)


Iteração:  1
Nome do repositório:freeCodeCamp
Commits:  23684
Url:  https://github.com/freeCodeCamp/freeCodeCamp
Contributors:  3674
Atualizando data frame :)

Iteração:  2
Nome do repositório:996.ICU
Commits:  2904
Url:  https://github.com/996icu/996.ICU
Contributors:  579
Atualizando data frame :)

Iteração:  3
Nome do repositório:vue
Commits:  3024
Url:  https://github.com/vuejs/vue
Contributors:  276
Atualizando data frame :)

Iteração:  4
Nome do repositório:bootstrap
Commits:  18744
Url:  https://github.com/twbs/bootstrap
Contributors:  1078
Atualizando data frame :)

Iteração:  5
Nome do repositório:react
Commits:  11106
Url:  https://github.com/facebook/react
Contributors:  1302
Atualizando data frame :)

Iteração:  6
Nome do repositório:tensorflow
Commits:  59510
Url:  https://github.com/tensorflow/tensorflow
Contributors:  2055
Atualizando data frame :)

Iteração:  7
Nome do repositório:free-programming-books
Commits:  5017
Url:  https://github.com/EbookFoundation/free-progr

Nome do repositório:JavaGuide
Commits:  1518
Url:  https://github.com/Snailclimb/JavaGuide
Contributors:  79
Atualizando data frame :)

Iteração:  56
Nome do repositório:reveal.js
Commits:  2341
Url:  https://github.com/hakimel/reveal.js
Contributors:  246
Atualizando data frame :)

Iteração:  57
Nome do repositório:socket.io
Commits:  1715
Url:  https://github.com/socketio/socket.io
Contributors:  153
Atualizando data frame :)

Iteração:  58
Nome do repositório:awesome-vue
Commits:  2748
Url:  https://github.com/vuejs/awesome-vue
Contributors:  1495
Atualizando data frame :)

Iteração:  59
Nome do repositório:30-seconds-of-code
Commits:  4480
Url:  https://github.com/30-seconds/30-seconds-of-code
Contributors:  172
Atualizando data frame :)

Iteração:  60
Nome do repositório:build-your-own-x
Commits:  355
Url:  https://github.com/danistefanovic/build-your-own-x
Contributors:  53
Atualizando data frame :)

Iteração:  61
Nome do repositório:Semantic-UI
Commits:  6684
Url:  https://githu

Commits:  1835
Url:  https://github.com/nvm-sh/nvm
Contributors:  243
Atualizando data frame :)

Iteração:  110
Nome do repositório:nw.js
Commits:  3625
Url:  https://github.com/nwjs/nw.js
Contributors:  97
Atualizando data frame :)

Iteração:  111
Nome do repositório:hacker-scripts
Commits:  138
Url:  https://github.com/NARKOZ/hacker-scripts
Contributors:  41
Atualizando data frame :)

Iteração:  112
Nome do repositório:bulma
Commits:  1313
Url:  https://github.com/jgthms/bulma
Contributors:  268
Atualizando data frame :)

Iteração:  113
Nome do repositório:interviews
Commits:  436
Url:  https://github.com/kdn251/interviews
Contributors:  29
Atualizando data frame :)

Iteração:  114
Nome do repositório:protobuf
Commits:  6715
Url:  https://github.com/protocolbuffers/protobuf
Contributors:  550
Atualizando data frame :)

Iteração:  115
Nome do repositório:normalize.css
Commits:  360
Url:  https://github.com/necolas/normalize.css
Contributors:  36
Atualizando data frame :)

Iteração:  1

Nome do repositório:pm2
Commits:  4545
Url:  https://github.com/Unitech/pm2
Contributors:  
Atualizando data frame :)

Iteração:  163
Nome do repositório:grafana
Commits:  22222
Url:  https://github.com/grafana/grafana
Contributors:  895
Atualizando data frame :)

Iteração:  164
Nome do repositório:pytorch
Commits:  19071
Url:  https://github.com/pytorch/pytorch
Contributors:  1094
Atualizando data frame :)

Iteração:  165
Nome do repositório:nocode
Commits:  3
Url:  https://github.com/kelseyhightower/nocode
Contributors:  1
Atualizando data frame :)

Iteração:  166
Nome do repositório:AdminLTE
Commits:  1017
Url:  https://github.com/ColorlibHQ/AdminLTE
Contributors:  117
Atualizando data frame :)

Iteração:  167
Nome do repositório:tech-interview-handbook
Commits:  170
Url:  https://github.com/yangshun/tech-interview-handbook
Contributors:  54
Atualizando data frame :)

Iteração:  168
Nome do repositório:gin
Commits:  1276
Url:  https://github.com/gin-gonic/gin
Contributors:  221
Atua

Nome do repositório:deeplearningbook-chinese
Commits:  727
Url:  https://github.com/exacity/deeplearningbook-chinese
Contributors:  
Atualizando data frame :)

Iteração:  217
Nome do repositório:cpython
Commits:  104741
Url:  https://github.com/python/cpython
Contributors:  925
Atualizando data frame :)

Iteração:  218
Nome do repositório:Leaflet
Commits:  6744
Url:  https://github.com/Leaflet/Leaflet
Contributors:  618
Atualizando data frame :)

Iteração:  219
Nome do repositório:frp
Commits:  699
Url:  https://github.com/fatedier/frp
Contributors:  35
Atualizando data frame :)

Iteração:  220
Nome do repositório:phaser
Commits:  13114
Url:  https://github.com/photonstorm/phaser
Contributors:  416
Atualizando data frame :)

Iteração:  221
Nome do repositório:awesome-wechat-weapp
Commits:  515
Url:  https://github.com/justjavac/awesome-wechat-weapp
Contributors:  
Atualizando data frame :)

Iteração:  222
Nome do repositório:incubator-superset
Commits:  4243
Url:  https://github.com/ap

...

Commits:  2006
Url:  https://github.com/pcottle/learnGitBranching
Contributors:  
Atualizando data frame :)

Iteração:  761
Nome do repositório:cypress
Commits:  11964
Url:  https://github.com/cypress-io/cypress
Contributors:  108
Atualizando data frame :)

Iteração:  762
Nome do repositório:amazeui
Commits:  1279
Url:  https://github.com/amazeui/amazeui
Contributors:  
Atualizando data frame :)

Iteração:  763
Nome do repositório:mustache.js
Commits:  736
Url:  https://github.com/janl/mustache.js
Contributors:  91
Atualizando data frame :)

Iteração:  764
Nome do repositório:js-cookie
Commits:  576
Url:  https://github.com/js-cookie/js-cookie
Contributors:  
Atualizando data frame :)

Iteração:  765
Nome do repositório:vuepress
Commits:  1630
Url:  https://github.com/vuejs/vuepress
Contributors:  217
Atualizando data frame :)

Iteração:  766
Nome do repositório:Public-APIs
Commits:  414
Url:  https://github.com/n0shake/Public-APIs
Contributors:  
Atualizando data frame :)

Iteração:  

Contributors:  
Atualizando data frame :)

Iteração:  815
Nome do repositório:leetcode
Commits:  228
Url:  https://github.com/azl397985856/leetcode
Contributors:  14
Atualizando data frame :)

Iteração:  816
Nome do repositório:cobra
Commits:  707
Url:  https://github.com/spf13/cobra
Contributors:  156
Atualizando data frame :)

Iteração:  817
Nome do repositório:IntelliJ-IDEA-Tutorial
Commits:  430
Url:  https://github.com/judasn/IntelliJ-IDEA-Tutorial
Contributors:  
Atualizando data frame :)

Iteração:  818
Nome do repositório:cayley
Commits:  1344
Url:  https://github.com/cayleygraph/cayley
Contributors:  75
Atualizando data frame :)

Iteração:  819
Nome do repositório:30-seconds-of-css
Commits:  388
Url:  https://github.com/30-seconds/30-seconds-of-css
Contributors:  
Atualizando data frame :)

Iteração:  820
Nome do repositório:Blog
Commits:  194
Url:  https://github.com/mqyqingfeng/Blog
Contributors:  
Atualizando data frame :)

Iteração:  821
Nome do repositório:gotty
Commits: 

Contributors:  2
Atualizando data frame :)

Iteração:  869
Nome do repositório:pouchdb
Commits:  4584
Url:  https://github.com/pouchdb/pouchdb
Contributors:  
Atualizando data frame :)

Iteração:  870
Nome do repositório:100-Days-Of-ML-Code
Commits:  295
Url:  https://github.com/MLEveryday/100-Days-Of-ML-Code
Contributors:  
Atualizando data frame :)

Iteração:  871
Nome do repositório:Chameleon
Commits:  342
Url:  https://github.com/viccalexander/Chameleon
Contributors:  
Atualizando data frame :)

Iteração:  872
Nome do repositório:svgo
Commits:  979
Url:  https://github.com/svg/svgo
Contributors:  121
Atualizando data frame :)

Iteração:  873
Nome do repositório:WaveFunctionCollapse
Commits:  100
Url:  https://github.com/mxgmn/WaveFunctionCollapse
Contributors:  13
Atualizando data frame :)

Iteração:  874
Nome do repositório:googletest
Commits:  2851
Url:  https://github.com/google/googletest
Contributors:  223
Atualizando data frame :)

Iteração:  875
Nome do repositório:Mock
Comm

Nome do repositório:Signal-Android
Commits:  4318
Url:  https://github.com/signalapp/Signal-Android
Contributors:  183
Atualizando data frame :)

Iteração:  925
Nome do repositório:create-react-native-app
Commits:  259
Url:  https://github.com/react-community/create-react-native-app
Contributors:  49
Atualizando data frame :)

Iteração:  926
Nome do repositório:bootstrap-datepicker
Commits:  1636
Url:  https://github.com/uxsolutions/bootstrap-datepicker
Contributors:  310
Atualizando data frame :)

Iteração:  927
Nome do repositório:Motrix
Commits:  502
Url:  https://github.com/agalwood/Motrix
Contributors:  24
Atualizando data frame :)

Iteração:  928
Nome do repositório:chartist-js
Commits:  853
Url:  https://github.com/gionkunz/chartist-js
Contributors:  66
Atualizando data frame :)

Iteração:  929
Nome do repositório:matomo
Commits:  25833
Url:  https://github.com/matomo-org/matomo
Contributors:  227
Atualizando data frame :)

Iteração:  930
Nome do repositório:node_redis
Commits: 

Nome do repositório:stetho
Commits:  579
Url:  https://github.com/facebook/stetho
Contributors:  
Atualizando data frame :)

Iteração:  978
Nome do repositório:playframework
Commits:  9548
Url:  https://github.com/playframework/playframework
Contributors:  701
Atualizando data frame :)

Iteração:  979
Nome do repositório:FizzBuzzEnterpriseEdition
Commits:  161
Url:  https://github.com/EnterpriseQualityCoding/FizzBuzzEnterpriseEdition
Contributors:  
Atualizando data frame :)

Iteração:  980
Nome do repositório:goreplay
Commits:  992
Url:  https://github.com/buger/goreplay
Contributors:  
Atualizando data frame :)

Iteração:  981
Nome do repositório:Effeckt.css
Commits:  432
Url:  https://github.com/h5bp/Effeckt.css
Contributors:  
Atualizando data frame :)

Iteração:  982
Nome do repositório:Mantle
Commits:  1212
Url:  https://github.com/Mantle/Mantle
Contributors:  
Atualizando data frame :)

Iteração:  983
Nome do repositório:mybatis-3
Commits:  2919
Url:  https://github.com/mybatis/

In [151]:
resultados_csv.tail()

,id,owner_type,owner_url,owner_html_url,html_url,url,fork,created_at,updated_at,size,...,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,timestamp_extract,commits,contributors
1015,45896779,Organization,https://api.github.com/users/jikexueyuanwiki,https://github.com/jikexueyuanwiki,https://github.com/jikexueyuanwiki/tensorflow-zh,https://api.github.com/repos/jikexueyuanwiki/t...,False,2015-11-10T07:59:30Z,2019-07-06T08:10:34Z,44266,...,TeX,True,True,4090,4090,28,10977,1562435119,714.0,
1016,58327877,Organization,https://api.github.com/users/browsh-org,https://github.com/browsh-org,https://github.com/browsh-org/browsh,https://api.github.com/repos/browsh-org/browsh,False,2016-05-08T19:34:15Z,2019-07-06T16:21:52Z,7371,...,JavaScript,True,True,308,308,108,10972,1562435119,372.0,18
1017,43482568,Organization,https://api.github.com/users/jobbole,https://github.com/jobbole,https://github.com/jobbole/awesome-java-cn,https://api.github.com/repos/jobbole/awesome-j...,False,2015-10-01T06:53:06Z,2019-07-06T16:17:25Z,257,...,NaN,True,True,3879,3879,1,10971,1562435119,239.0,
1018,63935685,User,https://api.github.com/users/gztchan,https://github.com/gztchan,https://github.com/gztchan/awesome-design,https://api.github.com/repos/gztchan/awesome-d...,False,2016-07-22T08:06:02Z,2019-07-06T17:22:52Z,241,...,NaN,True,True,881,881,8,10968,1562435119,194.0,
1019,29900673,User,https://api.github.com/users/phanan,https://github.com/phanan,https://github.com/phanan/htaccess,https://api.github.com/repos/phanan/htaccess,False,2015-01-27T06:25:41Z,2019-07-05T01:35:41Z,211,...,NaN,True,True,1144,1144,6,10968,1562435119,156.0,


# Verificando dados extraídos

In [160]:
contributors_faill = resultados_csv.loc[resultados_csv['contributors'] == '']
contributors_faill

,id,owner_type,owner_url,owner_html_url,html_url,url,fork,created_at,updated_at,size,...,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,timestamp_extract,commits,contributors
30,460078,Organization,https://api.github.com/users/angular,https://github.com/angular,https://github.com/angular/angular.js,https://api.github.com/repos/angular/angular.js,False,2010-01-06T00:34:37Z,2019-07-06T01:50:47Z,100204,...,JavaScript,True,True,28880,28880,461,59570,1562435054,8964.0,
38,14098069,User,https://api.github.com/users/justjavac,https://github.com/justjavac,https://github.com/justjavac/free-programming-...,https://api.github.com/repos/justjavac/free-pr...,False,2013-11-04T01:59:19Z,2019-07-06T16:42:32Z,848,...,NaN,True,False,16955,16955,164,51994,1562435054,885.0,
76,1334369,Organization,https://api.github.com/users/resume,https://github.com/resume,https://github.com/resume/resume.github.com,https://api.github.com/repos/resume/resume.git...,False,2011-02-06T13:39:55Z,2019-07-06T17:23:08Z,411,...,JavaScript,True,True,1095,1095,35,41258,1562435057,263.0,
88,4311796,User,https://api.github.com/users/adam-p,https://github.com/adam-p,https://github.com/adam-p/markdown-here,https://api.github.com/repos/adam-p/markdown-here,False,2012-05-13T03:27:22Z,2019-07-06T17:41:29Z,16214,...,JavaScript,True,True,6701,6701,242,39044,1562435057,807.0,
90,101296881,Organization,https://api.github.com/users/mtdvio,https://github.com/mtdvio,https://github.com/mtdvio/every-programmer-sho...,https://api.github.com/repos/mtdvio/every-prog...,False,2017-08-24T13:18:26Z,2019-07-06T16:20:09Z,153,...,NaN,True,True,3555,3555,28,38963,1562435058,230.0,
96,156018,User,https://api.github.com/users/antirez,https://github.com/antirez,https://github.com/antirez/redis,https://api.github.com/repos/antirez/redis,False,2009-03-21T22:32:25Z,2019-07-06T16:11:50Z,77140,...,C,True,False,14376,14376,2285,37365,1562435058,8174.0,
105,23974149,User,https://api.github.com/users/Dogfalo,https://github.com/Dogfalo,https://github.com/Dogfalo/materialize,https://api.github.com/repos/Dogfalo/materialize,False,2014-09-12T19:35:38Z,2019-07-06T14:58:43Z,164708,...,JavaScript,True,True,4804,4804,679,36051,1562435058,3866.0,
115,211666,Organization,https://api.github.com/users/nodejs,https://github.com/nodejs,https://github.com/nodejs/node-v0.x-archive,https://api.github.com/repos/nodejs/node-v0.x-...,False,2009-05-27T16:29:46Z,2019-07-06T12:14:12Z,144238,...,NaN,True,True,7779,7779,571,35462,1562435059,2.0,
118,9185792,Organization,https://api.github.com/users/apache,https://github.com/apache,https://github.com/apache/incubator-echarts,https://api.github.com/repos/apache/incubator-...,False,2013-04-03T03:18:59Z,2019-07-06T16:33:42Z,155232,...,JavaScript,True,True,10528,10528,3082,35137,1562435059,5275.0,
119,107111421,User,https://api.github.com/users/thedaviddias,https://github.com/thedaviddias,https://github.com/thedaviddias/Front-End-Chec...,https://api.github.com/repos/thedaviddias/Fron...,False,2017-10-16T10:12:36Z,2019-07-06T16:37:41Z,3182,...,JavaScript,True,False,3375,3375,5,34920,1562435059,662.0,


In [167]:
resultados_csv.isNan()

AttributeError: 'DataFrame' object has no attribute 'isNan'

Pegando readme

In [ ]:
repo = g.get_repo("freeCodeCamp")
contents = repo.get_contents("README.md")
print(contents.decoded_content)